In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Required Imports

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Model
from keras.applications.densenet import DenseNet121
from keras.applications.inception_v3 import InceptionV3

Downloading the DermaMNIST dataset

In [ ]:
!pip install medmnist
from medmnist import DermaMNIST
dataset_train = DermaMNIST(split="train", download=True)
dataset_validate = DermaMNIST(split="val", download=True)
dataset_test = DermaMNIST(split="test", download=True)

Using downloaded and verified file: /root/.medmnist/dermamnist.npz
Using downloaded and verified file: /root/.medmnist/dermamnist.npz
Using downloaded and verified file: /root/.medmnist/dermamnist.npz


Shape of original images

In [ ]:
x_train, y_train = dataset_train.imgs, dataset_train.labels
x_test, y_test = dataset_test.imgs, dataset_test.labels
x_val, y_val = dataset_validate.imgs, dataset_validate.labels

print(x_train.shape, y_train.shape, x_train[0].shape)

(7007, 28, 28, 3) (7007, 1) (28, 28, 3)


Preprocessing the images

In [ ]:
import cv2
import numpy as np

def preprocess_images(images):
    preprocessed_images = []
    for image in images:
        resized_image = cv2.resize(image, (192, 192))
        preprocessed_images.append(resized_image)
    return np.array(preprocessed_images)

In [ ]:
resized_x_train = preprocess_images(x_train)
resized_x_test = preprocess_images(x_test)
resized_x_val = preprocess_images(x_val)
resized_x_train.shape, resized_x_test.shape, resized_x_val.shape

((7007, 192, 192, 3), (2005, 192, 192, 3), (1003, 192, 192, 3))

In [ ]:
#The path has to be updated accordingly - Put the correct path to pre-trained DenseNet121 and InceptionV3 models here
model1 = tf.keras.models.load_model('/content/DermaMNIST_DenseNet121.h5')
model2 = tf.keras.models.load_model('/content/DermaMNIST_InceptionV3.h5')

In [ ]:
num_samples = len(resized_x_test)
num_classes = 7

In [ ]:
print(num_samples)

2005


In [ ]:
resized_x_test.shape

(2005, 192, 192, 3)

Taking the predictions made by different models into account

In [ ]:
predictions = np.zeros((len(resized_x_test), num_classes))

model_predictions1 = model1.predict(resized_x_test)
predictions += model_predictions1
model_predictions2 = model2.predict(resized_x_test)
predictions += model_predictions2

63/63 [==============================] - 7s 66ms/step


In [ ]:
model_predictions1.shape
model_predictions2.shape

(2005, 7)

Product Rule Ensemble

In [ ]:
# Apply product rule ensemble
ensemble_predictions_product_rule = np.multiply(model_predictions1, model_predictions2)
ensemble_predictions_pr = np.argmax(ensemble_predictions_product_rule, axis=1)
from sklearn.metrics import accuracy_score
y_true = y_test
accuracy = accuracy_score(y_true, ensemble_predictions_pr)
print("Product Rule Ensemble Accuracy:", accuracy)

Product Rule Ensemble Accuracy: 0.7945137157107232


In [ ]:
from sklearn.metrics import classification_report

classification_report_pr = classification_report(y_true, ensemble_predictions_pr)

print(classification_report_pr)

              precision    recall  f1-score   support

           0       0.49      0.55      0.51        66
           1       0.57      0.70      0.63       103
           2       0.61      0.50      0.55       220
           3       0.75      0.13      0.22        23
           4       0.64      0.38      0.47       223
           5       0.87      0.95      0.90      1341
           6       0.90      0.66      0.76        29

    accuracy                           0.79      2005
   macro avg       0.69      0.55      0.58      2005
weighted avg       0.78      0.79      0.78      2005

